In [16]:
import itertools as iter
import numpy as np
import math
import random as random

In [17]:
#Mergesort on lists
def merge_sort(lists):
    if len(lists) <= 1:
        return lists
    m = int(len(lists) / 2)
    left_lists = lists[:m]
    right_lists = lists[m:]
    left_lists = merge_sort(left_lists)
    right_lists = merge_sort(right_lists)
    return merge(left_lists, right_lists)
def merge(l_list, r_list):
    merged = []
    l, r = 0,0
    order = len(l_list[0])
    while (l < len(l_list)) and (r < len(r_list)):
        if l_list[l] < r_list[r]:
            merged.append(l_list[l])
            l+=1
        else:
            merged.append(r_list[r])
            r+=1
    if l < len(l_list):
        while l < len(l_list):
            merged.append(l_list[l])
            l+=1
    else:
        while r < len(r_list):
            merged.append(r_list[r])
            r+=1
    return merged

l = [[2,3,4],[1,2,3]]
merge_sort(l)

[[1, 2, 3], [2, 3, 4]]

In [18]:
def all_sets(J,k):
    if len(J) < k:
        return "Ya Done Goofed"
    return [list(x) for x in iter.combinations(J, k)]

def good_in_packet(x,y,packets):
    packet = []
    x.sort()
    y.sort()
    if np.all(x == y):
        raise ValueError('Inputs are the same')
    for a in packets:
        if x in packets[a] and y in packets[a]:
            packet = a
    return packet

In [19]:
def flip(exp, indices):
    copy = [k for k in exp]
    start = indices[0]
    end = indices[1]
    for i in range(0,end-start+1):
        copy[start+i] = exp[end-i]
    return copy

In [20]:
def move_up(topexp, elt,packets):
    return ((np.sum([len(good_in_packet(x,elt,packets))>0 for x in topexp])) == 0)

#Tries to bubble down the first word given by expression[indices]
#returns a new expression with the first word bubbled down as far as it will go (up until the final index in indices)
#returns successful = True if first word is bubbled below the end index
def bubble_down(packet, head_elt, subexp,packets):
    bubbled = False
    cur_pos = 0
    end_pos = len(subexp) - 1
    while cur_pos < end_pos:
        #print('The head element is {}. And the subexpression is {}'.format(head_elt, subexp))
        #print('This is the current position {}. This is the end position {}'.format(cur_pos,end_pos))
        next_pos = cur_pos + 1
        next_elt = subexp[next_pos]
        if move_up([head_elt], next_elt,packets):
            subexp = flip(subexp,[cur_pos, next_pos])
            cur_pos += 1
            if next_elt == packet[-1]:
                end_pos -= 1
            #print('The head element successfully moved down past {}\n'.format(next_elt))
            continue
        else:
            if next_elt in packet:
                #print('The head element is blocked by the packet element {}. \nThe head_elt is: {} The next elt is {}\n'.format(head_elt,next_elt, subexp))
                return False, subexp
            else:
                #print('The head element is blocked by a non-packet element.\n Head element is {}. And next_elt is {}\n'.format(head_elt, next_elt))
                bubbled, subsubexp = bubble_down(packet, next_elt, subexp[next_pos:end_pos+1],packets)
                subexp[next_pos:end_pos+1] = subsubexp
                if bubbled == True:
                    end_pos -=1
                else:
                    return False, subexp
    return True, subexp

In [21]:
#takes in a subexpression and a packet
#tells you if the packet can be brought together via equivalences
#if you can then it will return the new word
#At the start exp begins with the first element in packet and ends with the last element in packet
def come_together(exp, packet,packets):
   # print(exp, packet)
    assert(len(exp) >= len(packet)),'Your packet is bigger than the expression'
    pointer = 0 #<--- Keeps track of the leading packet element that has been encountered
    cur_pos = 0 #<--- Keeps track of where the head_packet_elt is
    end_pos = len(exp) - 1 #<--- Keeps track of where the last packet element is in the expression
    
    if len(exp) == len(packet):
    #    print('This is your expression {}\nThis is your packet {}'.format(exp,packet))
    #    print('The packet is flippable')
        return True, exp, end_pos
    
    while True:
    #    print('This is your expression {}\nThis is your packet {}'.format(exp,packet))
    #    print('This is the end position {}\nThis is the current position {}\nThis is the pointer {}'.format(
    #        end_pos, cur_pos, pointer, len(packet)))
        if (end_pos - (cur_pos - pointer)) + 1 == len(packet): #So the packet has been brought together
    #        print('Your packet has been brought together. This is the end position {} and this is the start position {}'.format(end_pos, cur_pos))
            break
        next_pos = cur_pos + 1
        next_elt = exp[next_pos]
    #    print(cur_pos, next_pos)
        if next_elt in packet:
            pointer += 1
            cur_pos += 1
            #if cur_pos == end_pos-1:
            #    print('Your packet has been brought together. This is the end position {} and this is the start position {}'.format(
            #    end_pos, cur_pos))
            continue
        else:
            if move_up(packet[:pointer+1], next_elt,packets) == True: #y can be moved above the packet elements above it
                del exp[next_pos]
                exp.insert(cur_pos-pointer,next_elt) #This was wrong. You don't always move up all the way. Just before the seen packet elements
                cur_pos += 1
                continue
            else: #so y can't be bubbled up, let's try bubbling it down
    #            print('You cannot bubble up {} from {}'.format(next_elt, exp))
                bubbled, subexp = bubble_down(packet, next_elt, exp[next_pos:end_pos+1],packets)
                if bubbled == True:
    #                print('You successfully bubbled {} out of {}'.format(next_elt, exp))
                    exp[next_pos:end_pos+1] = subexp
                    end_pos = exp.index(packet[-1]) 
    #                print('The new end position is {}\nThe new expression is {}'.format(end_pos,exp))
    #                print('The pointer is {} and the current position is {}'.format(pointer, cur_pos))
    #                print('The gap is {}'.format(end_pos - (cur_pos - pointer)))
                    continue
                else:
                    return False, exp, end_pos
    return True, exp, end_pos

#packets are dictionaries containing the k+1 sets along with the 
#consecutive pairings of k-sets in the packet when ordered in the lex order
def find_exp_node_flips(expression, packets):
    next_words = {}
    split_pairs = []
    for p in packets: #Should also check if p is in order
        skip = False
        for x in packets[p]:
            if x in split_pairs:
                skip = True
                continue
        if skip == True:
            continue
        else:
            packet = packets[p]
            try:
                indices = [expression.index(x) for x in packet]
            except ValueError:
                skip == True
            else:
                if sorted(indices) != indices:
                    skip == True
                    continue
                #print('This is the expression: {}\nThis is the packet: {}\nHere are the indices {}'.format(
                #    expression, packet, indices))
                #print('The packet has length {}'.format(len(packet)))
                #print('The subexpression is {}'.format(expression[indices[0]:indices[-1]+1]))
                brought_together, exp, end_pos = come_together(expression[indices[0]:indices[-1]+1],packet, packets)
                if brought_together == True:
                    exp = flip(exp,[end_pos - len(packet) +1,end_pos])
                    next_exp = expression.copy()
                    next_exp[indices[0]:indices[-1]+1] = exp
                #    print('The packet was flipped\n')
                    next_words[p] = next_exp
                else:
                    #print('The packet was not flipped\n')
                    pass
    return next_words

In [22]:
#Checks if the cur_word's history has been seen before.
#If it has, then it returns the node with the same history
#cur_word_hist: list of tuples
#checked_hist: dictionary whose index is the level (path length) and entries are a [[node1:[tuples]],[node2: [tuples1],[tuples2]]]
def check_history(cur_word_hist, checked_hist):
    cur_level = len(cur_word_hist[0])
    cur_word_hist = set(cur_word_hist[0])
    tracker = 0
    if cur_level not in checked_hist.keys():
        return False, -1
    
#     print(cur_level, cur_word_hist)
    
    for i in checked_hist[cur_level]:
        check_set = set(i)
#         print(check_set)
        if cur_word_hist == check_set:
            spot = np.sum([len(checked_hist[j]) for j in range(0,cur_level)]) + tracker + 1
            return True, spot
        tracker += 1
    return False, -1

def build_p_history(word, word_p_hist, total_p_hist, parent_node):
    history = [x.copy() for x in total_p_hist[parent_node].copy()]
    history = [x + [word_p_hist] for x in history]
    return history

In [23]:
def build_condensed_graph(n_sets, start_word, add_flips = True):
    count = 0
    checked_words = [start_word]
    unchecked_words = []
    checked_history = {0: []}
    f_connections = {0: []}
    p_connections = {0: []}
    p_history = {0: []}
    n_connections = {0: []}
    n_history = {0: []}
    equiv_words = {}
    
    connections = find_exp_node_flips(start_word,n_sets)
    f_moves = [connections[w] for w in connections]
    p_moves = list(connections.keys())
    for f in range(0,len(f_moves)):
        level = 1
        unchecked_words.append(f_moves[f])
        i = len(checked_words) + len(unchecked_words) - 1
        f_connections[i] = []
        p_connections[i] = []
        n_connections[i] = []
        n_history[i] = [[len(checked_words) - 1]]
        p_history[i] = [[p_moves[f]]]             
        j = len(checked_words) - 1
        f_connections[j].append(f_moves[f])
        p_connections[j].append(p_moves[f])
        n_connections[j].append(i)
        if level in checked_history.keys():
             checked_history[level].append(p_history[i][0])
        else:
             checked_history[level] = [p_history[i][0]]
    while len(unchecked_words) != 0:
        #print(count)
        cur_word = unchecked_words.pop(0)
        checked_words.append(cur_word)
        add_node_num = len(checked_words) - 1
        if count > 500:
            break
        else:
            connections = find_exp_node_flips(cur_word,n_sets)
            f_moves = [connections[w] for w in connections]
            p_moves = list(connections.keys())
            l = len(f_moves)
            for f in range(0,len(f_moves)):
                poss_word = f_moves[f]
                poss_p_history = build_p_history(poss_word, p_moves[f], p_history, add_node_num)
                level = len(poss_p_history[0])
                found, index = check_history(poss_p_history, checked_history)
                #Almost there. Just need to actually use check_history
                if found == False:
                    #print("The word has not been seen yet. Checked history returned: {}".format(found))
                    unchecked_words.append(poss_word)
                    i = len(checked_words) + len(unchecked_words) - 1
                    f_connections[i] = []
                    p_connections[i] = []
                    n_connections[i] = []
                    n_history[i] = [x.copy() for x in n_history[add_node_num].copy()]
                    n_history[i] = [x+[add_node_num] for x in n_history[i]]
                    p_history[i] = poss_p_history
                    j = add_node_num #checked_words.index(cur_word)
                    f_connections[j].append(poss_word)
                    p_connections[j].append(p_moves[f])
                    n_connections[j].append(i)
                    if level in checked_history.keys():
                        checked_history[level].append(poss_p_history[0])
                    else:
                        checked_history[level] = [poss_p_history[0]]
                else:
                    #print("The word has been seen already. Checked history returned: {}".format(found))
#                      index = unchecked_words.index(f_moves[f]) + len(checked_words)
                    history = n_history[add_node_num].copy()
                    history = [x.copy() for x in history]
                    history = [x+[add_node_num] for x in history]
                    for r in history:
                        n_history[index].append(r)
                    history = p_history[add_node_num].copy()
                    history = [x.copy() for x in history]
                    history = [r+[p_moves[f]] for r in history]
                    for r in history:
                        p_history[index].append(r)
                    j = add_node_num
                    f_connections[j].append(f_moves[f])
                    p_connections[j].append(p_moves[f])
                    n_connections[j].append(index)
        #count += 1
    return checked_words, f_connections, p_connections, p_history, n_connections, n_history

In [24]:
def get_packets_dict(J,k):
    packets = {}
    for x in iter.combinations(J,k+1):
        packets[x] = [list(y) for y in iter.combinations(list(x),k)]
    return packets

def generate_graph(J,k, condensed = True, lex_word = None):
    if lex_word == None:
        lex_word = [list(x) for x in all_sets(J,k)]
    packets = get_packets_dict(J,k)       
    if condensed == True:
        graph = build_condensed_graph(packets, lex_word)
    else:
        return "You haven't made this"
        #graph = quick_build_graph(one_packets,two_packets,lex_word)
    return graph
        
def visualize_graph(graph, forward, packets, output = None):
    if output != None:
        file = open(output, 'w')
    level = 0
    counter = 0
    if forward == True:
        n = 4
        p = 2
    else:
        n = 5
        p = 3
    if output != None:
        file.write('Here are the level 0 nodes: \n')
    else:
        print('Here are the level 0 nodes: ')
    for i in graph[-1]:
        n_history_set = graph[n][i]
        p_history_set = graph[p][i]
        if i != 0:
            if len(graph[3][i][-1]) > level:
                level += 1
                if output != None:
                    file.write('There were {} level {} nodes.\n \nHere are the level {} nodes\n'.format(counter, level -1, level))
                else:
                    print('There were {} level {} nodes.\n \nHere are the level {} nodes'.format(counter, level -1, level))
                counter = 0
        if output != None:
            file.write("{}: {}\n".format(i,n_history_set))
        else:
            print("{}: {}".format(i,n_history_set))
        if packets == True:
            if output != None:
                file.write("{}: {}\n".format(i, p_history_set))
            else:
                print("{}: {}".format(i, p_history_set))
        counter += 1
    if output != None:
        file.close()

In [25]:
#B_4 k = 3 Playground
n = 5
k = 4
J = [1,2,3,4,5,6]

lex_word = [list(x) for x in iter.combinations(J,k)]
#lex_word = [[1],[2],[3],[4],[5]]

k_4_3_graph = generate_graph(J,k,lex_word=lex_word)
packets = get_packets_dict(J,k)
visualize_graph(k_4_3_graph, forward = True, packets = True, output = None )

Here are the level 0 nodes: 
0: [1, 2]
0: [(1, 2, 3, 4, 5), (2, 3, 4, 5, 6)]
There were 1 level 0 nodes.
 
Here are the level 1 nodes
1: [3]
1: [(1, 2, 3, 4, 6)]
2: [4]
2: [(1, 3, 4, 5, 6)]
There were 2 level 1 nodes.
 
Here are the level 2 nodes
3: [5]
3: [(1, 2, 3, 5, 6)]
4: [6]
4: [(1, 2, 4, 5, 6)]
There were 2 level 2 nodes.
 
Here are the level 3 nodes
5: [7]
5: [(1, 2, 4, 5, 6)]
6: [8]
6: [(1, 2, 3, 5, 6)]
There were 2 level 3 nodes.
 
Here are the level 4 nodes
7: [9]
7: [(1, 3, 4, 5, 6)]
8: [10]
8: [(1, 2, 3, 4, 6)]
There were 2 level 4 nodes.
 
Here are the level 5 nodes
9: [11]
9: [(2, 3, 4, 5, 6)]
10: [11]
10: [(1, 2, 3, 4, 5)]
There were 2 level 5 nodes.
 
Here are the level 6 nodes
11: []
11: []


In [91]:
random.seed(0)
def gen_realizable_set(J,k, steps):
    packets = get_packets_dict(J,k-1)
    lex_word = [list(x) for x in iter.combinations(J,k-1)]
    real_set = []
    for i in range(steps):
        moves = find_exp_node_flips(lex_word,packets)
        rand_step = random.sample(moves.keys(),1)[0]
        lex_word = moves[rand_step]
        real_set.append(rand_step)
    return real_set

#Takes in a realizable k set and returns J_s, J_F, J_0, J_p
def find_k_plus_sets(real,k):
    real_0, real_s, real_p, real_F = [],[],[],[]
    packets = get_packets_dict(J,k)
    for x in packets:
        p = [tuple(y) for y in packets[x]]
        intersection = set(p) & set(real)
        if len(intersection) == 0:
            real_0.append(x)
        elif len(intersection) == len(packets[x]):
            real_F.append(x)
        elif tuple(packets[x][0]) in intersection:
            real_p.append(x)
        elif tuple(packets[x][-1]) in intersection:
            real_s.append(x)
    print('The following packets are empty in Real: {}'.format(real_0))
    print('The following packets are suffixes in Real: {}'.format(real_s))
    print('The following packets are prefixes in Real: {}'.format(real_p))
    print('The following packets are full in Real: {}\n'.format(real_F))
    return real_0,real_s,real_p,real_F

J = [1,2,3,4,5,6]
k = 3
packets = get_packets_dict(J,k)
#l_word = lex_word
#order = [(2,3,4),(1,3,4),(1,2,4),(2,3,5),(1,3,5),(1,2,5),(2,3,6),(1,3,6),(1,2,6),(1,2,3)]
#order.reverse()
#for o in order:
#    l_word = find_exp_node_flips(l_word, packets)[o]
#find_exp_node_flips(find_exp_node_flips(find_exp_node_flips(l_word,packets)[(1,2,3)],packets)[(3,4,5)],packets)
#find_exp_node_flips(lex_word,packets)[]
#find_exp_node_flips(l_word,packets)

real = gen_realizable_set(J,k,11)
missing = set(get_packets_dict(J,k-1).keys()) - set(real)
r_0, r_s, r_p, r_F = find_k_plus_sets(real,k)

print('You are working with {} \n\nIt is missing {}'.format(real, missing))


The following packets are empty in Real: [(1, 2, 4, 5)]
The following packets are suffixes in Real: [(1, 2, 3, 4), (1, 2, 3, 5), (1, 2, 3, 6), (1, 2, 4, 6), (1, 2, 5, 6), (1, 4, 5, 6), (2, 4, 5, 6), (3, 4, 5, 6)]
The following packets are prefixes in Real: [(1, 3, 4, 5), (1, 3, 4, 6), (2, 3, 4, 5), (2, 3, 4, 6)]
The following packets are full in Real: [(1, 3, 5, 6), (2, 3, 5, 6)]

You are working with [(4, 5, 6), (2, 3, 4), (1, 3, 4), (2, 3, 5), (2, 3, 6), (1, 3, 5), (1, 3, 6), (2, 5, 6), (1, 5, 6), (2, 4, 6), (3, 5, 6)] 

It is missing {(3, 4, 6), (1, 2, 6), (1, 2, 3), (1, 4, 5), (2, 4, 5), (1, 2, 4), (3, 4, 5), (1, 4, 6), (1, 2, 5)}


In [92]:
get_packets_dict(J,4)

{(1, 2, 3, 4, 5): [[1, 2, 3, 4],
  [1, 2, 3, 5],
  [1, 2, 4, 5],
  [1, 3, 4, 5],
  [2, 3, 4, 5]],
 (1, 2, 3, 4, 6): [[1, 2, 3, 4],
  [1, 2, 3, 6],
  [1, 2, 4, 6],
  [1, 3, 4, 6],
  [2, 3, 4, 6]],
 (1, 2, 3, 5, 6): [[1, 2, 3, 5],
  [1, 2, 3, 6],
  [1, 2, 5, 6],
  [1, 3, 5, 6],
  [2, 3, 5, 6]],
 (1, 2, 4, 5, 6): [[1, 2, 4, 5],
  [1, 2, 4, 6],
  [1, 2, 5, 6],
  [1, 4, 5, 6],
  [2, 4, 5, 6]],
 (1, 3, 4, 5, 6): [[1, 3, 4, 5],
  [1, 3, 4, 6],
  [1, 3, 5, 6],
  [1, 4, 5, 6],
  [3, 4, 5, 6]],
 (2, 3, 4, 5, 6): [[2, 3, 4, 5],
  [2, 3, 4, 6],
  [2, 3, 5, 6],
  [2, 4, 5, 6],
  [3, 4, 5, 6]]}